In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Load dataset
# Dataset berupa "heart.csv", pastikan file ada di direktori kerja
# Dataset ini mengandung informasi mengenai penyakit jantung (biner klasifikasi)
data = pd.read_csv("/content/drive/MyDrive/Dataset/heart.csv")

# Pisahkan fitur (X) dan label (y)
X = data.drop(columns=["target"])
y = data["target"]

# Bagi dataset menjadi data latih dan data uji (80% latih, 20% uji)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalisasi data untuk memastikan semua fitur memiliki skala yang sama
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Konversi data ke bentuk tensor PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Fungsi untuk membangun MLP model
def create_mlp(input_size, hidden_layers, activation_fn):
    layers = []
    # Tambahkan lapisan input -> hidden pertama
    prev_size = input_size
    for hidden_size in hidden_layers:
        layers.append(nn.Linear(prev_size, hidden_size))
        layers.append(activation_fn)
        prev_size = hidden_size
    # Tambahkan lapisan output (1 unit untuk klasifikasi biner)
    layers.append(nn.Linear(prev_size, 2)) # 2 kelas output
    return nn.Sequential(*layers)

# Fungsi untuk pelatihan dan evaluasi model
def train_and_evaluate(model, optimizer, criterion, X_train, y_train, X_test, y_test, epochs, batch_size):
    # Gunakan DataLoader untuk batch training
    dataset = torch.utils.data.TensorDataset(X_train, y_train)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epochs):
        model.train()  # Set model ke mode pelatihan
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()  # Reset gradien
            predictions = model(batch_X)  # Prediksi output
            loss = criterion(predictions, batch_y)  # Hitung loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update bobot

    # Evaluasi model
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)
        y_pred_classes = torch.argmax(y_pred, dim=1)
        accuracy = accuracy_score(y_test, y_pred_classes)
    return accuracy

# Hyperparameter yang ingin diuji
hidden_layers_list = [[4], [8, 4], [16, 8, 4]]  # Kombinasi hidden layer
activation_functions = [nn.Identity(), nn.Sigmoid(), nn.ReLU(), nn.Tanh()]
epochs_list = [1, 10, 25, 50, 100, 250]
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64, 128, 256, 512]

input_size = X_train.shape[1]
results = []

# Lakukan eksperimen dengan semua kombinasi
for hidden_layers in hidden_layers_list:
    for activation_fn in activation_functions:
        for epochs in epochs_list:
            for lr in learning_rates:
                for batch_size in batch_sizes:
                    # Inisialisasi model, optimizer, dan loss function
                    model = create_mlp(input_size, hidden_layers, activation_fn)
                    optimizer = optim.Adam(model.parameters(), lr=lr)
                    criterion = nn.CrossEntropyLoss()

                    # Latih dan evaluasi model
                    accuracy = train_and_evaluate(model, optimizer, criterion, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, epochs, batch_size)

                    # Simpan hasil
                    results.append({
                        "hidden_layers": hidden_layers,
                        "activation_function": activation_fn.__class__.__name__,
                        "epochs": epochs,
                        "learning_rate": lr,
                        "batch_size": batch_size,
                        "accuracy": accuracy
                    })

# Simpan hasil ke dalam DataFrame untuk analisis lebih lanjut
results_df = pd.DataFrame(results)
print(results_df.sort_values(by="accuracy", ascending=False).head(10))

# Simpan hasil ke file CSV
results_df.to_csv("mlp_experiments_results.csv", index=False)


     hidden_layers activation_function  epochs  learning_rate  batch_size  \
2364    [16, 8, 4]                ReLU     250          0.001          16   
2366    [16, 8, 4]                ReLU     250          0.001          64   
2359    [16, 8, 4]                ReLU     250          0.010          32   
1279        [8, 4]             Sigmoid     250          0.010          32   
2361    [16, 8, 4]                ReLU     250          0.010         128   
2362    [16, 8, 4]                ReLU     250          0.010         256   
2363    [16, 8, 4]                ReLU     250          0.010         512   
2502    [16, 8, 4]                Tanh      50          0.010          16   
2365    [16, 8, 4]                ReLU     250          0.001          32   
2574    [16, 8, 4]                Tanh     250          0.010          16   

      accuracy  
2364       1.0  
2366       1.0  
2359       1.0  
1279       1.0  
2361       1.0  
2362       1.0  
2363       1.0  
2502       1.0  

In [6]:
import os
print(os.getcwd())

/content


In [7]:
results_df.to_csv('/content/drive/MyDrive/Dataset/mlp_experiments_results.csv', index=False)